In [1]:
import pandas as pd
import torch as th
from torch.utils.data import DataLoader, Dataset
import albumentations as A
from albumentations.pytorch import ToTensorV2

from solution.dstl_detection.dstl_processing import DstlProcessingLib

In [2]:
MODEL_NAME = "dstl_model"
LOCATION = "models/dstl"
DSTL_PATH = "/Users/cristianion/Desktop/satimg_data/DSTL"
TRAIN_WKT_FILE = DSTL_PATH + "/train_wkt_v4.csv"
GRID_SIZES_FILE = DSTL_PATH + "/grid_sizes.csv"

IMAGES_MULTICHANNEL = DSTL_PATH + "/sixteen_band"
IMAGES_RGB = DSTL_PATH + "/three_band"

# Column names
COL_MULTIPOLYGONWKT = "MultipolygonWKT"
COL_CLASSTYPE = "ClassType"
COL_IMAGEID = "ImageId"
COL_XMAX = "Xmax"
COL_YMIN = "Ymin"

# Image extension
EXT_TIFF = ".tif"


# resize parameters
IMAGE_RES_X = 512
IMAGE_RES_Y = 512


CLASSES = ["building", "structures", "road", "track", "tree", "crops", "waterway", "standing_water", "vehicle_large", "vehicle_small"]


In [3]:
class DstlDataset(Dataset):
    def __init__(self, transform) -> None:
        super().__init__()

        df_wkt = pd.read_csv(TRAIN_WKT_FILE)
        df_gs = pd.read_csv(GRID_SIZES_FILE, names=['ImageId', 'Xmax', 'Ymin'], skiprows=1)

        # self.df_samples = df_wkt
        self.transform = transform
        self.processing = DstlProcessingLib(df_wkt, df_gs, CLASSES)
        self.df_wkt = df_wkt

    def __len__(self):
        return len(self.df_wkt)

    def __getitem__(self, index):
        sample = self.df_wkt.iloc[index]

        image, mask = self.processing.read_image_and_mask(
            raster_size=(IMAGE_RES_Y, IMAGE_RES_X),
            image_id=sample[COL_IMAGEID],
        )
        mask = mask[0]

        if self.transform:
            augmentations = self.transform(image=image)
            image = augmentations["image"]

        return image, mask

In [4]:
DSTL_TRAIN_TRANSFORM = A.Compose([
    A.Resize(height=IMAGE_RES_Y, width=IMAGE_RES_X),
    A.Normalize(
        mean=[0.0, 0.0, 0.0],
        std=[1.0, 1.0, 1.0],
        max_pixel_value=255.0,
    ),
    ToTensorV2(),
])

DSTL_VAL_TRANSFORM = A.Compose([
    A.Resize(height=IMAGE_RES_Y, width=IMAGE_RES_X),
    A.Normalize(
        mean=[0.0, 0.0, 0.0],
        std=[1.0, 1.0, 1.0],
        max_pixel_value=255.0,
    ),
    ToTensorV2(),
])

In [5]:
dataset = DstlDataset(DSTL_TRAIN_TRANSFORM)

In [6]:
dataset[0]

(tensor([[[1.2588, 1.2902, 1.2980,  ..., 1.4196, 1.4471, 1.4392],
          [1.3059, 1.3804, 1.3059,  ..., 1.4078, 1.4314, 1.4431],
          [1.3333, 1.3686, 1.2627,  ..., 1.4118, 1.4196, 1.4275],
          ...,
          [1.2627, 1.3020, 1.2784,  ..., 1.2863, 1.2667, 1.2510],
          [1.2588, 1.2902, 1.2588,  ..., 1.3137, 1.2980, 1.2824],
          [1.2353, 1.2588, 1.2667,  ..., 1.2235, 1.2745, 1.3098]],
 
         [[1.6000, 1.6392, 1.6549,  ..., 1.7176, 1.7294, 1.7137],
          [1.6392, 1.7059, 1.6588,  ..., 1.7098, 1.7137, 1.7216],
          [1.6549, 1.6863, 1.6000,  ..., 1.7176, 1.7216, 1.7216],
          ...,
          [1.5804, 1.6039, 1.6078,  ..., 1.6235, 1.6078, 1.5922],
          [1.5843, 1.6078, 1.6157,  ..., 1.6549, 1.6392, 1.6157],
          [1.5725, 1.6000, 1.6275,  ..., 1.5647, 1.6118, 1.6471]],
 
         [[1.1922, 1.2157, 1.2275,  ..., 1.2392, 1.2353, 1.2314],
          [1.2078, 1.2471, 1.2196,  ..., 1.2392, 1.2431, 1.2392],
          [1.2118, 1.2353, 1.1882,  ...,